## Network Link-to-Link attribute Validation Checks

Functions used to identify Link-to-Link pairs and compare values. This ncludes:
- Single Attribute In-Link checks
- Single Attribute In-Link / Out-Link checks
- Multiple Attribute In-Link / Out-Link checks

#### Import Libraries 

In [1]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
# from nxviz import CircosPlot
from shapely.geometry import LineString

### Load Network data

#### Create Function: *segment_heading*: 
Calculate Heading for both Initial segment and Final segment of Link

In [2]:
def segment_heading(line, first_segment=True, flip=False):
    if flip:
        line = LineString(line.coords[::-1])
    
    segment = line.coords[:2] if first_segment else line.coords[-2:]
    
    from_pt = segment[0]
    to_pt = segment[1]
    
    y_delta = to_pt[1] - from_pt[1]
    x_delta = to_pt[0] - from_pt[0]

    angle_radians = np.arctan2(y_delta, x_delta)
    if angle_radians < 0:
        angle_radians = 2 * np.pi + angle_radians;

    # Deal with the angles being from East Counterclockwise to Northerly bearings
    degrees = 90 - angle_radians * 180 / np.pi
    if degrees < 0:
        degrees = degrees + 360

    return degrees

#### Load Node and Line Network data from Shapefiles
Use GeoPandas to load Geospatial data  
Call function 'segment_heading' to calculate Heading

In [3]:
line_shp = gpd.read_file(os.path.join('..', 'data', 'Chatt_Master.shp')).to_crs(epsg=2274)
node_shp = gpd.read_file(os.path.join('..', 'data', 'Chatt_Master_Node.shp')).to_crs(epsg=2274)

# Heading AB
line_shp['AB_START_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=True)
line_shp['AB_END_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=False)
# Heading BA
line_shp['BA_START_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=True, flip=True)
line_shp['BA_END_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=False, flip=True)

### Build Network Graph

#### Turn Two-Way Links into 2 One-Way Links for Directional Flow.
#### Create Function: *directional_links*
Keep AB Links and produce BA Links, carrying over link attributes based on the direction of flow.

In [4]:
def directional_links(atts, line_shp):
    dirs = {'keep':'AB', 'invert':'BA'}
    dfs = {}
    
    for flow,d in dirs.items():
        links = line_shp[line_shp['{}_LANES'.format(d)] > 0].copy()
        rename_dict = {'{}{}'.format(d, a):a for a in atts}
        links = links.rename(columns=rename_dict)
        
        if flow == 'invert':
            links['A'] = links['TO_ID']
            links['B'] = links['FROM_ID']
            links = links.drop(columns=['FROM_ID', 'TO_ID']).rename(columns={'A': 'FROM_ID', 'B': 'TO_ID'})
        
        dfs['{}_links'.format(d)] = links
    
    links = pd.concat(dfs.values(), ignore_index=True)
    
    drop_dict = ['{}{}'.format(d, a) for a in atts for d in dirs.values()]
    links = links.drop(columns=drop_dict)
    
    return links

#### Generate Directional links with Attributes
####  Call Function: *directional_links*
Use list of Directional Attributes to carry over, e.g. 'AB_LANES' and 'BA_LANES', to '_LANES' 

In [5]:
# List of Directional Attributes
atts = [
    '_LINKDIR',
    '_LINKDTR',
    '_LANES',
    '_PARKING',
    '_TRIMS',
    '_AADT',
    '_CAR_ADT',
    '_SUT_ADT',
    '_MUT_ADT',
    '_BASEVOL',
    '_AFFTIME',
    '_AFFSPD',
    '_UCDELAY',
    '_DLYCAP',
    '_AMCAP',
    '_PMCAP',
    '_BPRA',
    '_BPRB',
    '_START_HEADING',
    '_END_HEADING'
]
# Generate Directional Links
line_shp = directional_links(atts, line_shp)

####  Build the Directional Graph: *g*
#### Call Method: *nx.from_pandas_edgelist*

In [6]:
g = nx.from_pandas_edgelist(
    line_shp,
    source='FROM_ID',
    target='TO_ID',
    edge_attr=True,
    create_using=nx.DiGraph()
)

assert nx.is_directed(g)

### Link to Link validation Functions

#### Create Function: *link_matcher_atts*: 
For each node in Network, match all combinations of In/ Out Links and return both In and Out values

In [7]:
def link_matcher_atts(attributes, graph, edge_id='ID'):
    heading = True if '_HEADING' in attributes else False
    atts = []
    atts.extend([a for a in attributes if a!='_HEADING'])
    
#     if heading: atts.remove('_HEADING')

    node_feed = []

    for node in node_shp['ID']:
        if node not in g:
            print('Node not in network: {}'.format(node))
            continue

        for i_o, i_d, i_data  in g.in_edges(node, data=True):
            for o_o, o_d, o_data in g.out_edges(node, data=True):
                if i_o == o_d and i_d == o_o:
                    # This is just a U-Turn on the "same" link
                    continue
                    
                data = [i_data, o_data]
                values = [node, *[d[edge_id] for d in data], *[d[a] for a in atts for d in data]]
                if heading: values.extend([i_data['_END_HEADING'], o_data['_START_HEADING']])
                node_feed.append(values)
                
    # Generate Column names
    dirs = ['in', 'out']
    cols = ['node', *[ d+'_'+edge_id for d in dirs], *[ d+'_'+a for a in atts for d in dirs]]
    if heading: cols.extend([ d+'__HEADING' for d in dirs])

    return pd.DataFrame(node_feed, columns=cols)

#### Create Function: *attribute_change*: 
Calculate change in Attribute

In [8]:
def attribute_change(network, attributes):
    for att in attributes:
        in_fld = 'in_{}'.format(att)
        out_fld = 'out_{}'.format(att)
        fld_diff = 'diff_{}'.format(att)
#         abs_fld_diff = 'abs_diff_{}'.format(filter_name)
    
        if isinstance(network[in_fld][0], (int, float)):
            network[fld_diff] = network[out_fld] - network[in_fld]
        elif isinstance(network[in_fld][0], str):
            network[fld_diff] = 'SAME' if network[out_fld].str == network[in_fld].str else 'CHANGES'

    return network#.groupby([in_fld, out_fld]).count().reset_index()

#### Create Function: *attribute_filtering*: 
Apply Attributes and Values difference to use as filters

In [9]:
def attribute_filtering(network, att_filters):
    for filter_name, filter_val in att_filters.items():
        in_fld = 'in_{}'.format(filter_name)
        out_fld = 'out_{}'.format(filter_name)
        fld_diff = 'diff_{}'.format(filter_name)
#         abs_fld_diff = 'abs_diff_{}'.format(filter_name)
    
        if isinstance(filter_val, (int, float)):
            network = network[abs(network[fld_diff]) >= filter_val]
        elif isinstance(filter_val, str):
            network = network[(network[in_fld] == filter_val) | (network[out_fld] == filter_val)]

    return network#.groupby([in_fld, out_fld]).count().reset_index()

#### Create Function: *export_geo*: 
Export Geometry to Shapefile

In [10]:
# In/Out Long Table
def export_geo(network, export_name, att_name):
    fld_diff = 'diff_{}'.format(att_name)
    
    network = network.rename(columns={'out_ID':'ID'})

    # geometries
    geo_cols = ['ID', 'FROM_ID', 'geometry']
    network = pd.merge(line_shp[geo_cols], network, left_on=['ID', 'FROM_ID'], right_on=['ID', 'node'])
    network.drop(columns=['FROM_ID'], inplace=True)
    
    shp_export = os.path.join('..', 'data', '{}_{}.shp'.format(export_name, '_'.join([n for n in att_name])))
    network.to_file(shp_export)
#     return network#.sort_values(by=fld_diff, ascending=False), shp_export
    return network, shp_export

#### Create Function: *summary_in_out*: 
Create function to aggregate In-link and Out-Link by Attribute and summarize by count

In [11]:
def summary_in_out(network, att, csv_name, filter_in='', filter_out='', return_summary = True, include_same_att=False):
    in_att = 'in_{}'.format(*att)
    out_att = 'out_{}'.format(*att)
    
    if include_same_att == False:
        network = network[network[in_att]!=network[out_att]]
        
    if filter_in is not '': network = network[network[in_att].isin(filter_in)]
    if filter_out is not '': network = network[network[out_att].isin(filter_out)]
        
    if return_summary == True:
        network = network.groupby([in_att, out_att])[in_att].count().reset_index(name='count')
    
    csv_export = os.path.join('..', 'data', '{}_{}_SUMMARY.csv'.format(csv_name, *attribute_name))
    network.to_csv(csv_export, index=False)

    return network

#### Create Function: *link_compare*: 
Create function to call Link-to-Link attribute comparison

In [12]:
def link_compare(attribute_names, attribute_filters, export_geometry):
    network = link_matcher_atts(attribute_names, graph=g)
    network = attribute_change(network, attribute_names)
    network = attribute_filtering(network,
                                attribute_filters,
                                )
    csv_export = os.path.join('..', 'data', 'Network_attribute_change_table_{}.csv'.format('_'.join([n for n in attribute_names])))
    network.to_csv(csv_export, index=False)
    
    # Print
    print_txt = 'Total network links with change in {}: {:,}'
    print(print_txt.format(', '.join([n for n in attribute_names]),
                           len(network)
                           ))
    print('Table exported to {}'.format(csv_export))
    if export_geometry:
        network, shp_export = export_geo(network, attribute_filters)
        print('Shapefile exported to {}'.format(shp_export))

    return network

#### Create Function: *link_compare_single*: 
Create function to call Link-to-Link for single attribute comparison

In [16]:
### RUN FUNCTIONS
def link_compare_single(att_name, export_name, in_att_filters, out_att_filters, return_summary, include_same_att):
    network = link_matcher_atts(att_name, graph=g)

    network = summary_in_out(network,
                             att_name,
                             export_name,
                             in_att_filters,
                             out_att_filters,
                             return_summary,
                             include_same_att
                             )
    
    print_txt = 'Total network links found {}: {:,}'
    print(print_txt.format(*att_name, len(network)))
    print('Table exported to {}'.format(export_name))
    
    if return_summary == False:
        network, shp_export = export_geo(network, export_name, att_name)
        print('Shapefile exported to {}'.format(shp_export))
        
    return network#.astype(int)

## Run Network VALIDATION TESTS
Use Heading ('_HEADING') to calculate deviation of In-Link(Final Segment) and Out-Link (Initial Segment)


## Single Attribute

### Zone Connectors to Freeway
### Node Link-IN  Link-OUT Summary validation: FUNCTIONAL CLASS

In [21]:
### SPECIFY PARAMETERS
# IN Attribute:
attribute_name = ['FUNCCLASS']
# Filter by OUT attribute value (Equal to):
in_attribute_filters = [98]
out_attribute_filters = [1,11,12]
# Return Summary table or Link level table
return_summary = False
# Include Links where Attribute remains the same In/Out
include_same_att = False
#Export name:
export_name = 'Network_attribute_change'


### RUN FUNCTIONS
network = link_compare_single(attribute_name,
                              export_name,
                              in_attribute_filters,
                              out_attribute_filters,
                              return_summary,
                              include_same_att
                              )

network

Total network links found FUNCCLASS: 6
Table exported to Network_attribute_change_OUT
Shapefile exported to ..\data\Network_attribute_change_OUT_FUNCCLASS.shp


,ID,geometry,node,in_ID,in_FUNCCLASS,out_FUNCCLASS
0,38590,"LINESTRING (2268824.838 292236.760, 2259832.56...",2327,261,11.0,98.0
1,38568,"LINESTRING (2122049.554 220243.903, 2126226.47...",16801,17644,1.0,98.0
2,38568,"LINESTRING (2122049.554 220243.903, 2126226.47...",16801,17648,1.0,98.0
3,38572,"LINESTRING (2122579.151 239648.417, 2127434.61...",16651,17520,11.0,98.0
4,38598,"LINESTRING (2264082.271 188253.464, 2258664.68...",16903,17769,1.0,98.0
5,38262,"LINESTRING (2171785.002 273318.099, 2172061.66...",2120,80,12.0,98.0


###  Node Link-IN  Link-OUT Summary validation: NUMBER OF LANES

In [ ]:
### SPECIFY PARAMETERS
# IN Attribute:
attribute_name = ['_LANES']
# Filter by OUT attribute value (Equal to):
in_attribute_filters = ''
out_attribute_filters = [5]
# Return Summary table or Link level table
return_summary = True
# Include Links where Attribute remains the same In/Out
include_same_att = False
#Export to csv name:
csv_name = 'Network_attribute_change_table'


### RUN FUNCTIONS
network = link_compare_single(network,
                              attribute_name,
                              csv_name,
                              in_attribute_filters,
                              out_attribute_filters,
                              return_summary,
                              include_same_att
                              )

network

###  Node Link-IN  Link-OUT Summary validation: SPEED LIMIT

In [ ]:
### SPECIFY PARAMETERS
# IN Attribute:
attribute_name = ['SPD_LMT']
# Filter by OUT attribute value (Equal to):
in_attribute_filters = ''
out_attribute_filters = [70]
# Return Summary table or Link level table
return_summary = True
# Include Links where Attribute remains the same In/Out
include_same_att = False
#Export to csv name:
csv_name = 'Network_attribute_change'


### RUN FUNCTIONS
network = link_compare_single(network,
                              attribute_name,
                              csv_name,
                              in_attribute_filters,
                              out_attribute_filters,
                              return_summary,
                              include_same_att
                              )

network

###  Node Link-IN  Link-OUT Summary validation: FUNCTIONAL CLASS

In [ ]:
### SPECIFY PARAMETERS
# IN Attribute:
attribute_name = ['FUNCCLASS']
# Filter by IN / OUT attribute value (Equal to):
in_attribute_filters = [98]
out_attribute_filters = [1, 11, 12]
# Return Summary table or Link level table
return_summary = False
# Include Links where Attribute remains the same In/Out
include_same_att = False
#Export to csv name:
csv_name = 'Network_attribute_change_table'


### RUN FUNCTIONS
network = link_compare_single(network,
                              attribute_name,
                              csv_name,
                              in_attribute_filters,
                              out_attribute_filters,
                              return_summary,
                              include_same_att
                              )

network

###  Node Link-IN  Link-OUT Summary validation: RIGHT SHOULDER WIDTH

In [ ]:
### SPECIFY PARAMETERS
# IN Attribute:
attribute_name = ['RS_WIDTH']
# Filter by IN / OUT attribute value (Equal to):
in_attribute_filters = ''
out_attribute_filters = [24, 30]
# Return Summary table or Link level table
return_summary = True
# Include Links where Attribute remains the same In/Out
include_same_att = False
#Export to csv name:
csv_name = 'Network_attribute_change_table'


### RUN FUNCTIONS
network = link_compare_single(network,
                              attribute_name,
                              csv_name,
                              in_attribute_filters,
                              out_attribute_filters,
                              return_summary,
                              include_same_att
                              )

network

###  Node Link-IN  Link-OUT Summary validation: (EXCESSIVE) SPEED LIMITS

In [ ]:
### SPECIFY PARAMETERS
# IN Attribute:
attribute_name = ['SPD_LMT']
# Filter by IN / OUT attribute value (Equal to):
in_attribute_filters = ''
out_attribute_filters = [65, 70]
# Return Summary table or Link level table
return_summary = True
# Include Links where Attribute remains the same In/Out
include_same_att = False
#Export to csv name:
csv_name = 'Network_attribute_change_table'


### RUN FUNCTIONS
network = link_compare_single(network,
                              attribute_name,
                              csv_name,
                              in_attribute_filters,
                              out_attribute_filters,
                              return_summary,
                              include_same_att
                              )

network

## Multiple Attribute

### Node Link-IN  Link-OUT validation: SPEED LIMIT, and NUMBER OF LANES

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', '_LANES']
# Filter by CHANGE in attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':30,
#     '_LANES':1
}
# Summarize by Node Maximum value and Export Geo:
export_geometry = True


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, export_geometry)

### ADDITIONAL FILTERING
network[network['node'].isin([2137, 2663])].drop(columns=['geometry'])

### Node Link-IN  Link-OUT validation: SPEED LIMIT, NUMBER OF LANES and HEADING

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', '_LANES', '_HEADING']
# Filter by CHANGE in attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':25,
#     '_LANES':1,
#     '_HEADING':30
}
# Summarize by Node Maximum value and Export Geo:
export_geometry = True


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, export_geometry)
network.drop(columns=['geometry'])

# ### ADDITIONAL FILTERING
network[abs(network['diff__HEADING'])<=30].drop(columns=['geometry'])

### Network Link-to-Link attribute side-by-side validation: NUMBER OF LANES and FUNCTION CLASS

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['_LANES', 'FUNCCLASS', 'FUNC_CLASS']
# Filter by CHANGE in attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    '_LANES':3,
}
# Summarize by Node Maximum value and Export Geo:
export_geometry = True


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, export_geometry)
network.head()


### ADDITIONAL FILTERING
# network[network['node'].isin([15248])].drop(columns=['geometry', 'diff_FUNCCLASS'])

### ADDITIONAL FILTERING: None is IN/OUT FREEWAY
network[(~network['in_FUNCCLASS'].isin([1,11,12])) & (~network['out_FUNCCLASS'].isin([1,11,12]))]\
       .drop(columns=['geometry', 'diff_FUNCCLASS'])

### Network Link-to-Link attribute side-by-side validation: SPEED LIMIT and ROUTE NAME
Filter by Speed Limit

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', 'RTE_NME']
# Filter by CHANGE in attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':10,
}
# Summarize by Node Maximum value and Export Geo:
export_geometry = False


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, export_geometry)
network.head()

### Network Link-to-Link attribute side-by-side validation: SPEED LIMIT and ROUTE NAME
Filter by Route Name

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', 'RTE_NME']
# Filter by CHANGE in attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':10,
    'RTE_NME':'CENTROID CONNECTOR',
}
# Summarize by Node Maximum value and Export Geo:
export_geometry = False


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, export_geometry)
network.head()

### Network Link-to-Link attribute side-by-side validation: NUMBER OF LANES and FUNCTIONAL CLASS
Filter by Number of Lanes

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['_LANES', 'FUNCCLASS']
# Filter by CHANGE in attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    '_LANES':2,
}
# Summarize by Node Maximum value and Export Geo:
export_geometry = True


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, export_geometry)
network.head()